In [24]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus

path = './baseball_reference_2016_clean.csv'
pd_data = pd.read_csv('./baseball_reference_2016_clean.csv')

In [25]:
pd_data = pd_data.drop('Unnamed: 0', axis = 1)
pd_data = pd_data.drop('attendance', axis = 1)
pd_data = pd_data.drop('away_team', axis = 1)
pd_data = pd_data.drop(['date', 'field_type', 'game_type', 'home_team'], axis = 1)
pd_data = pd_data.drop(['start_time', 'venue', 'day_of_week', 'temperature'], axis = 1)
pd_data = pd_data.drop(['wind_speed', 'wind_direction', 'sky', 'total_runs'], axis = 1)
pd_data = pd_data.drop(['game_hours_dec', 'season', 'home_team_loss', 'home_team_outcome'], axis = 1)
pd_data

,away_team_errors,away_team_hits,away_team_runs,home_team_errors,home_team_hits,home_team_runs,home_team_win
0,1,7,3,0,9,4,1
1,0,5,2,0,8,3,1
2,0,5,2,0,9,4,1
3,0,8,3,1,8,1,0
4,1,8,4,0,8,3,0
5,1,11,10,1,7,2,0
6,0,9,2,1,7,3,1
7,0,6,3,1,2,0,0
8,1,8,5,2,12,6,1
9,0,11,5,0,10,4,0


In [14]:
pd_data['away_team_runs'].mean()

4.413723101908242

In [22]:
# answer
train = pd_data[:2000]
ans = train['home_team_win'].values
train = train.drop('home_team_win', axis = 1)
# test data
test = pd_data[2000:]
test_ans = test['home_team_win'].values
test = test.drop('home_team_win', axis = 1)

In [19]:
dtree = DecisionTreeClassifier(max_depth = 4)
dtree.fit(train, ans)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file = dot_data,  
                filled = True, 
                feature_names = list(train),
                class_names = ['lose','win'],
                special_characters = True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

In [20]:
dtree.feature_importances_


array([0.        , 0.        , 0.50825778, 0.        , 0.        ,
       0.49174222])

In [21]:
predict = dtree.predict(test)
from sklearn.metrics import accuracy_score

accuracy_score(test_ans, predict)

0.9935205183585313